# Model III Galaxy Classifier (Spiral, Elliptical, Odd objects)

**Most of this code is made by the authors of the paper of Ghaderi et al. (2025) (https://iopscience.iop.org/article/10.3847/1538-4365/ada8ab) and taken from the GitHub Repository: https://github.com/hmddev1/machine_learning_for_morphological_galaxy_classification**

**This notebook classifies the images of galaxies to spiral and elliptical galaxies and odd objects using the Vision Transformer and the CNN (Model III).**

**Purpose of this notebook: Run Model III for 10 iterations, and obtain results to reproduce table 4 of Ghaderi et al. (2025).**

In [ ]:
import os
path = '/content/drive/Shared drives/DLP Project/Project/Models/Galaxy Models/Looping each Model'
os.chdir(path)
%run imports.py
%matplotlib inline
import plotting
roc_curves = {}

# Paths to images
spath = r'/content/drive/Shared drives/DLP Project/Project/Data/galaxy/images/cropped_spiral'
epath = r'/content/drive/Shared drives/DLP Project/Project/Data/galaxy/images/cropped_elliptical'
opath = r'/content/drive/Shared drives/DLP Project/Project/Data/galaxy/images/cropped_odd'

# Default image size and zernike order.
image_size = 200
zernike_order = 45

# Loading the ZMs and concatenating to a consolidated dataset
spiral_data = pd.read_csv('/content/drive/Shared drives/DLP Project/Project/spiral_zms.csv')
elliptical_data = pd.read_csv('/content/drive/Shared drives/DLP Project/Project/elliptical_zms.csv')
odd_data = pd.read_csv('/content/drive/Shared drives/DLP Project/Project/odd_zms.csv')

spiral_data.drop("Unnamed: 0", axis = 1, inplace = True)
elliptical_data.drop("Unnamed: 0", axis = 1, inplace = True)
odd_data.drop("Unnamed: 0", axis = 1, inplace = True)

all_zm_data = np.concatenate([spiral_data, elliptical_data, odd_data])
np.shape(all_zm_data)

spiral_label = [0] * len(spiral_data)
elliptical_label = [1] * len(elliptical_data)
odd_label = [2] * len(odd_data)

all_labels = spiral_label + elliptical_label + odd_label
len(all_labels)

In [ ]:
def load_galaxy_images(data_dir, target_size):
        """
        Loads, resizes, and processes all JPG images from the specified directory.

        Parameters:
        data_dir (str): The directory containing the JPG images to be processed.
        target_size (tuple): The target size for resizing the images, specified as (width, height).

        Returns:
        list: A list of PIL Image objects, each representing a resized and processed image.

        The function performs the following steps:
        1. Lists all JPG image files in the specified directory.
        2. Reads each image using OpenCV.
        3. Resizes each image to the specified target size.
        4. Scales the pixel values and converts the image to a format compatible with PIL.
        5. Converts each resized image to a PIL Image object.
        6. Appends each PIL Image object to a list.
        7. Returns the list of PIL Image objects.
        """

        all_images = []

        file_path = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir) if filename.endswith('.jpg')]

        for img in file_path:
            image = cv2.imread(img)
            resized_images=cv2.resize(image, target_size)
            resized_images = (resized_images * 255).astype(np.uint8)
            pil_images = Image.fromarray(resized_images)
            all_images.append(pil_images)

        return all_images


sp_dir = '/content/drive/Shared drives/DLP Project/Project/Data/galaxy/images/cropped_spiral'
el_dir = '/content/drive/Shared drives/DLP Project/Project/Data/galaxy/images/cropped_elliptical'
odd_dir = '/content/drive/Shared drives/DLP Project/Project/Data/galaxy/images/cropped_odd'

image_size = 200

sp_img = load_galaxy_images(sp_dir, target_size=(image_size,image_size))
el_img = load_galaxy_images(el_dir, target_size=(image_size,image_size))
odd_img = load_galaxy_images(odd_dir, target_size=(image_size,image_size))

# Due to lack of computational power, we will use a subset of the data (6000 galaxies) while keeping the same ratios between categories.
sp_img = sp_img[:3136]
el_img = el_img[:2082]
odd_img = odd_img[:782]

all_data = sp_img + el_img + odd_img


# Labels
label_s = [0] * len(sp_img)
label_e = [1] * len(el_img)
label_o = [2] * len(odd_img)

all_labels = label_s + label_e + label_o
len(all_labels)


# transforms for training data
train_transform = transforms.Compose([transforms.CenterCrop(image_size),
                                      transforms.RandomRotation(90),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomResizedCrop(image_size, scale=(0.8, 1.0), ratio=(0.99, 1.01)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])

# transforms for test data
test_transform = transforms.Compose([transforms.CenterCrop(image_size),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])

In [ ]:
num_iterations = 10
metrics_list = []  # Store metrics for each iteration
all_roc_curves = []

for iteration in range(num_iterations):
    print(f"Iteration {iteration + 1}...")

    # Split data
    X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(all_data, all_labels, np.arange(len(all_labels)), test_size=0.25, shuffle=True, random_state=None)
    y_train_encoded = to_categorical(y_train, num_classes=3)

    # # Transformer for training data
    transformed_X_train = []
    for i in range(len(X_train)):
        transformed_train_images = train_transform(X_train[i])
        new_image = np.transpose(transformed_train_images, (1, 2, 0))
        transformed_X_train.append(new_image)

    # # Transformer for testing data
    transformed_X_test = []
    for j in range(len(X_test)):
        transformed_test_images = test_transform(X_test[j])
        new_images = np.transpose(transformed_test_images, (1, 2, 0))
        transformed_X_test.append(new_images)

    # Class weights
    class_weights = {0: len(all_data) / (3 * len(spiral_data)),
                     1: len(all_data) / (3 * len(elliptical_data)),
                     2: len(all_data) / (3 * len(odd_data))}

    # Building the model

    # input
    x = Input(shape=(image_size, image_size, 3))

    # hidden layers
    c0 = Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding="same")(x)
    b0 = BatchNormalization()(c0)
    m0 = MaxPooling2D(pool_size=(2, 2))(b0)
    d0 = Dropout(0.1)(m0)

    c1 = Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding="same")(m0)
    b1 = BatchNormalization()(c1)
    m1 = MaxPooling2D(pool_size=(2, 2))(b1)
    d1 = Dropout(0.1)(m1)

    c2 = Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding="same")(m1)
    b2 = BatchNormalization()(c2)
    m2 = MaxPooling2D(pool_size=(2, 2))(b2)
    d2 = Dropout(0.1)(m2)

    f = Flatten()(m2)

    de0 = Dense(64, activation='relu')(f)
    de1 = Dense(32, activation='relu')(de0)
    de2 = Dense(3, activation='softmax')(de1)

    model = Model(inputs=x, outputs=de2, name="cnn_transformer_galaxy_nonegalaxy")
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    optimizer = tf.keras.optimizers.Adam()
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

    # Training the model
    b_size = 64
    e_num = 30

    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

    history = model.fit(
        np.array(transformed_X_train), y_train_encoded,
        batch_size=b_size,
        epochs=e_num,
        verbose=1,
        class_weight=class_weights,
        callbacks=es,
        validation_split=0.1)

    y_pred = model.predict(np.array(transformed_X_test))
    y_pred_labels = np.argmax(y_pred, axis=1)

    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred_labels)

    # Performance metrics (per-class)
    recall_per_class = recall_score(y_test, y_pred_labels, average=None)
    precision_per_class = precision_score(y_test, y_pred_labels, average=None)
    f1_per_class = f1_score(y_test, y_pred_labels, average=None)
    accuracy = accuracy_score(y_test, y_pred_labels) # overall

    # Compute per-class TSS
    tss_per_class = {}
    for i, class_name in enumerate(['Spiral', 'Elliptical', 'Odd']):
        tp = cm[i, i]
        fn = np.sum(cm[i, :]) - tp
        fp = np.sum(cm[:, i]) - tp
        tn = np.sum(cm) - (tp + fn + fp)
        tss_per_class[class_name] = (tp / (tp + fn + 1e-6)) - (fp / (fp + tn + 1e-6))

    # Store metrics in a list
    metrics_list.append({
        'Iteration': iteration + 1,
        'Recall per Class': recall_per_class,
        'Precision per Class': precision_per_class,
        'F1 per Class': f1_per_class,
        'Accuracy': accuracy,
        'TSS per Class': tss_per_class
    })

for result in metrics_list:
    print(result)